In [1]:
import numpy as np
import pandas as pd
import pickle
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression as Log
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict, GridSearchCV

#### Load the data

In [12]:
# read the data with the non-text features
patents = pd.read_pickle("patent_data/nontext_features.p")

In [13]:
print patents.shape
patents.head()

(12030, 16)


,id,publication_year,B,C,D,E,F,G,H,num_applications,num_patent_citations,num_nonpatent_citations,num_claims,num_similar_doc,num_authors,payment_times
0,US6699658B1,2004,0,1,0,0,0,0,0,5,28,34,42,1,4,3
1,US6699724B1,2004,0,0,0,0,0,1,0,32,47,44,25,0,4,3
2,US6690816B2,2004,0,0,0,0,0,1,0,8,9,0,32,1,4,1
3,US6711436B1,2004,0,0,0,0,0,0,0,4,105,109,45,7,1,3
4,US6711432B1,2004,0,0,0,0,0,0,0,7,15,114,44,3,4,3


#### Format the data

In [14]:
# reformat the response variable into binary
y_data = np.zeros(patents.shape[0])
y_data[patents['payment_times'].values >= 2] = 1

print "Percentage of patents with > 1 maintenance fee payments: ", np.mean(y_data)

Percentage of patents with > 1 maintenance fee payments:  0.62859517872


In [15]:
# predictors
x_data = patents.drop(['id', 'payment_times'], axis = 1).values
x_data.shape

(12030, 14)

In [16]:
# split train and test sets
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, 
                                                    test_size = 0.2, 
                                                    random_state = 123)

print "Dataset dimensions:"
print "x_train: ", x_train.shape
print "x_test: ", x_test.shape
print "y_train: ", y_train.shape
print "y_test: ", y_test.shape

Dataset dimensions:
x_train:  (9624, 14)
x_test:  (2406, 14)
y_train:  (9624,)
y_test:  (2406,)


In [17]:
### subsampling the training data
# sample the same number of'useful' patents as the 'not useful' patents
# size of each class
num_size = np.sum(y_train == 0)

#random shuffle the rows
n = x_train.shape[0]
perm = range(n)
np.random.shuffle(perm)

x_train = x_train[perm]
y_train = y_train[perm]

# separate the two classes
x_useful = x_train[y_train == 1, :]
x_not_useful = x_train[y_train == 0, :]
y_useful = y_train[y_train == 1]
y_not_useful = y_train[y_train == 0]

# sample num_size from the 'useful' class
x_useful = x_useful[:num_size]
y_useful = y_useful[:num_size]

# combine the two classes
x_train_sub = np.concatenate((x_useful, x_not_useful), axis = 0)
y_train_sub = np.concatenate((y_useful, y_not_useful), axis = 0)

# shuffle again
# shuffle the combined data
n2 = x_train_sub.shape[0]
perm2 = range(n2)
np.random.shuffle(perm2)

x_train_sub = x_train_sub[perm2]
y_train_sub = y_train_sub[perm2]

# check the size
print x_train_sub.shape
print y_train_sub.shape

(7198, 14)
(7198,)


In [9]:
# # split train and test sets
# x_train, x_test, y_train, y_test = train_test_split(x_data_sub, y_data_sub, 
#                                                     test_size = 0.2, 
#                                                     random_state = 123)

# print "Dataset dimensions:"
# print "x_train: ", x_train.shape
# print "x_test: ", x_test.shape
# print "y_train: ", y_train.shape
# print "y_test: ", y_test.shape

Dataset dimensions:
x_train:  (7148, 14)
x_test:  (1788, 14)
y_train:  (7148,)
y_test:  (1788,)


### Model training
#### Basic Logistic Regression

In [20]:
### use logistic regression

# call the model function
model = Log()
# parameter tuning
c =  np.logspace(-5, 5, 11)

# use grid search with 5-fold CV
grid_model = GridSearchCV(model, param_grid = {'C': c}, cv  = 5, scoring = 'accuracy')
# fit on the data
grid_model = grid_model.fit(x_train_sub, y_train_sub) 

In [21]:
# check results
print "Best accuracy:", grid_model.best_score_
print "Best parameter: ", grid_model.best_params_

Best accuracy: 0.558627396499
Best parameter:  {'C': 0.01}


In [29]:
# check confusion matrix
best_log = grid_model.best_estimator_
best_log.fit(x_train_sub, y_train_sub)

y_pred = best_log.predict(x_test)

# accuracy
print "Test accuracy: ", np.mean(y_pred == y_test)
print "F1 score: ", metrics.f1_score(y_pred, y_test)
print "Precision: ", metrics.precision_score(y_pred, y_test)
print "Recall: ", metrics.recall_score(y_pred, y_test)
metrics.confusion_matrix(y_test, y_pred)

 test accuracy:  0.543640897756
F1 score:  0.585034013605
Precision:  0.50357839948
Recall:  0.697926059513


array([[534, 335],
       [763, 774]])

#### Logistic Regression with Balanced Weight

In [33]:
# call the model function with balanced weight
model = Log(class_weight='balanced')
# parameter tuning
c =  np.logspace(-5, 5, 11)

# use grid search with 5-fold CV
grid_model = GridSearchCV(model, param_grid = {'C': c}, cv  = 5, scoring = 'f1')
# fit on the data
grid_model = grid_model.fit(x_train, y_train) 

In [34]:
# check results
print "Best accuracy:", grid_model.best_score_
print "Best parameter: ", grid_model.best_params_

# check confusion matrix
# check confusion matrix
best_log = grid_model.best_estimator_
best_log.fit(x_train_sub, y_train_sub)

y_pred = best_log.predict(x_test)

# accuracy
print "Test accuracy: ", np.mean(y_pred == y_test)
print "F1 score: ", metrics.f1_score(y_pred, y_test)
print "Precision: ", metrics.precision_score(y_pred, y_test)
print "Recall: ", metrics.recall_score(y_pred, y_test)
metrics.confusion_matrix(y_test, y_pred)

Best accuracy: 0.595037700785
Best parameter:  {'C': 100.0}
Test accuracy:  0.544472152951
F1 score:  0.589820359281
Precision:  0.5126870527
Recall:  0.694273127753


array([[522, 347],
       [749, 788]])

#### Random Forest

In [35]:
### tune random forest

model = RandomForestClassifier(n_estimators = 100)

# tune max_features
param_space = np.arange(2, 15, 2)

grid_model = GridSearchCV(model, n_jobs = 4, 
                          param_grid = {'max_features': param_space}, 
                          cv  = 5, scoring = 'accuracy')
# fit on the data
grid_model = grid_model.fit(x_train_sub, y_train_sub)

In [36]:
# check results
print "Best accuracy:", grid_model.best_score_
print "Best parameter: ", grid_model.best_params_

Best accuracy: 0.549041400389
Best parameter:  {'max_features': 2}


In [37]:
# check confusion matrix
best_rf = grid_model.best_estimator_
best_rf.fit(x_train_sub, y_train_sub)

y_pred = best_rf.predict(x_test)

# accuracy
print "Test accuracy: ", np.mean(y_pred == y_test)
print "F1 score: ", metrics.f1_score(y_pred, y_test)
print "Precision: ", metrics.precision_score(y_pred, y_test)
print "Recall: ", metrics.recall_score(y_pred, y_test)
metrics.confusion_matrix(y_test, y_pred)

Test accuracy:  0.563175394846
F1 score:  0.617400800874
Precision:  0.551724137931
Recall:  0.700826446281


array([[507, 362],
       [689, 848]])

In [39]:
# pickle the rf model
pickle.dump(best_rf, open("models/best_rf.p", 'wb'))

In [41]:
# try open the model and predict
rf = pickle.load(open("models/best_rf.p", 'r'))

In [45]:
y_pred = rf.predict_proba(x_test[0].reshape((1, len(x_test[0]))))

In [47]:
y_pred[0]

array([ 0.34,  0.66])